In [13]:
import os
import re
from io import StringIO
from pathlib import Path
import pandas as pd
path_base = "C:\\Users\\16462\\OneDrive\\Desktop\\assignments\\PA\\Billboard\\billboard\\"

data_map = {}

start_year=2008
end_year=2014
# Get Data as [Year, DF] combo
for i in range (start_year, end_year):
    filename = path_base+str(i)+".csv"
    with open(filename) as file:
        lines = [line.encode('utf-8', errors='ignore').decode('utf-8') for line in file]
        df = pd.read_csv(StringIO('\n'.join(lines)), header=0)
        data_map[i] = df

In [23]:
# get relevant data from Spotify API for the same
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


client_id = '16a3911229cf4fc99361a96d524e6a99'
client_secret = '6038b52110c24d3fac28a5ef1178ad47'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

def generate_artists(artist_list):
    # Generate all possible artists with break points
    # feat. / comma / and / with
    return [x.strip() for x in re.split('feat.|and|with|,', artist_list)]

def handle_special_cases(artist, track):
    # Try and remove and search by each artist sequentially
    artist_list = generate_artists(artist_list=artist)
    for curr_artist in artist_list:
        track_id = sp.search(q='artist: '+ curr_artist +' track:' + track, type='track')
        base_data = track_id['tracks']['items']
        if(base_data):
                return base_data[0]['id']
    # Try without artist data as last straw
    track_id = sp.search(q=' track:' + track, type='track')
    base_data =  track_id['tracks']['items']
    if(not base_data):
        return 0
    return base_data[0]['id']

def generate_track_ids(df):
    final_lst = []
    for ind in df.index:
        artist = df['Artist'][ind]
        track =  df['Song Title'][ind]
        track_id = sp.search(q='artist: '+ artist +' track:' + track, type='track')
        base_data = track_id['tracks']['items']
        if(not base_data):
            final_lst.append(handle_special_cases(artist=artist, track=track))
            continue
        final_lst.append(base_data[0]['id'])
    return final_lst


def generate_components_from_track_ids(df):
    track_id_lst = df['spotify_track_ids']
    df_features = pd.DataFrame(columns= ['danceability', 'energy', 'speechiness', 'acousticness', 'liveness', 'instrumentalness', 'valence', 'tempo', 'loudness'])
    for id in track_id_lst:
        if(id == 0):
            dict_data = {'danceability' : 0, 'energy': 0,'speechiness': 0,'acousticness': 0, 'liveness': 0, 'instrumentalness': 0, 'valence': 0, 'tempo': 0, 'loudness': 0}
        else:
            base_features = sp.audio_features(id)[0]
            dict_data = {'danceability' : base_features['danceability'], 'energy': base_features['energy'],'speechiness': base_features['speechiness'],'acousticness': base_features['acousticness'], 'liveness': base_features['liveness'], 'instrumentalness': base_features['instrumentalness'], 'valence': base_features['valence'], 'tempo': base_features['tempo'], 'loudness': base_features['loudness']}
        df_features = pd.concat([df_features, pd.DataFrame.from_records([dict_data])])
    return df_features



def get_features_per_year(data_map):
    for i in range(start_year, end_year):
        base_df = data_map[i]
        base_df['spotify_track_ids'] = generate_track_ids(base_df)
        df_features = generate_components_from_track_ids(base_df)
        base_df['danceability'] = list(df_features['danceability'])
        base_df['energy'] = list(df_features['energy'])
        base_df['speechiness'] = list(df_features['speechiness'])
        base_df['acousticness'] = list(df_features['acousticness'])
        base_df['liveness'] = list(df_features['liveness'])
        base_df['instrumentalness'] = list(df_features['instrumentalness'])
        base_df['valence'] = list(df_features['valence'])
        base_df['tempo'] = list(df_features['tempo'])
        base_df['loudness'] = list(df_features['loudness'])
        data_map[i] = base_df

get_features_per_year(data_map=data_map)

In [ ]:
from lyricsgenius import Genius
from requests.exceptions import Timeout

lg_client_id = 'CQgAJwGPL9UggJTvrPNrHs-5Z7MsW_8NLB6Qf5wmMsdguul4G3wuXCwrMjPrFIbI'
lg_client_secret = '89fcCi1qp8s8ym1foi--p7-uGVOeO-1vKVQFWZAcB-SJNbTPqCNviWljpbvQmwqj6YOjIqKumjgIEVM4q6q5eA'
lg_access_token = 'hP5VBM1qM88JEGJwfl_NFD8Wv1bNo3nJZ5_zkis0McT2l4_wqqsfqKaNZLl-Kwa-'

genius = Genius(access_token=lg_access_token, excluded_terms=["(Remix)", "(Live)"], remove_section_headers=True, verbose=False)
genius.timeout = 1
genius.sleep_time = 1

def print_data_to_csv(data_map, year):
        df = data_map[year]
        filename = "test_"+str(year)+".csv"
        df.to_csv(filename, index=False)


def gen_lyics_for_songs(data_map):
    for i in range(start_year, end_year):
        song_lyrics_lst= list()
        df = data_map[i]
        for ind in df.index:
            artist = df['Artist'][ind]
            track_name = df['Song Title'][ind]
            retries = 0
            found_song = False
            while retries < 5 and found_song == False:
                try:
                    # Try and remove and search by each artist sequentially
                    artist_list = generate_artists(artist_list=artist)
                    for curr_artist in artist_list:
                        song = genius.search_song(track_name, artist=curr_artist)
                        if(song):
                            song_lyrics_lst.append(song.lyrics)
                            found_song = True
                        if(found_song == True):
                            break
                    if(found_song == True):
                        continue            
                    # Search song without artist, take the topmost
                    song = genius.search_song(track_name)
                    if(song):
                        song_lyrics_lst.append(song.lyrics)
                        found_song = True
                except Timeout as e:
                    retries+=1
                    continue
                retries+=1
            if(not song or found_song == False):
                song_lyrics_lst.append("")
                continue
        df['lyrics'] = song_lyrics_lst
        data_map[i] = df
        print_data_to_csv(data_map=data_map, year=i)



gen_lyics_for_songs(data_map=data_map)

In [53]:
from ibm_watson import NaturalLanguageUnderstandingV1
import json
import numpy as np
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features,CategoriesOptions,EmotionOptions,KeywordsOptions, ClassificationsOptions

authenticator = IAMAuthenticator('21B4ablXnI9YFEOcJOsNrF79HgNz2pR74-Q8vDBvyM8q')
tone_analyzer = NaturalLanguageUnderstandingV1(
    version='2022-04-07',
    authenticator=authenticator
)

tone_analyzer.set_service_url('https://api.us-south.natural-language-understanding.watson.cloud.ibm.com/instances/95ef2705-e234-45fa-8592-9d4f05544c00')
tone_analyzer.set_disable_ssl_verification(True)


def store_sentiment_dataframe(df, year):
        filename = "sentiment_"+str(year)+".csv"
        df.to_csv(filename, index=False)

def get_emotion(text, track_name):
    if(text==""):
        print(track_name," No data of lyrics")
        return {"anger": 0.0, "disgust": 0.0, "fear": 0.0, "joy": 0.0, "sadness": 0.0}
    json_output = tone_analyzer.analyze(text=text, features=Features(emotion=EmotionOptions(document=True)), language='en')
    if(json_output.get_status_code() == 400 or json_output.get_status_code() == 500):
        print(track_name,"Invalid language")
        return {"anger": 0.0, "disgust": 0.0, "fear": 0.0, "joy": 0.0, "sadness": 0.0}
    output = json_output.get_result()
    emotion_data = output['emotion']['document']['emotion']
    tones_dict = {"anger": emotion_data['anger'], "disgust": emotion_data['disgust'], "fear": emotion_data['fear'], "joy": emotion_data['joy'], "sadness": emotion_data['sadness']}
    return tones_dict


# Set songs for tone
def get_emotion_for_each_song(data_map, st_year, ed_year):
    for i in range(st_year, ed_year):
        df = data_map[i]
        df_features = pd.DataFrame(columns= ['anger', 'disgust', 'fear', 'joy', 'sadness'])
        for ind in df.index:
            lyrics = df["lyrics"][ind]
            df_lyric  = get_emotion(lyrics, df['Song Title'][ind])
            df_features = pd.concat([df_features, pd.DataFrame.from_records([df_lyric])])
        df['anger'] =  list(df_features['anger'])
        df['disgust'] =  list(df_features['disgust'])
        df['fear'] =  list(df_features['fear'])
        df['joy'] =  list(df_features['joy'])
        df['sadness'] =  list(df_features['sadness'])
        data_map[i] = df
        store_sentiment_dataframe(data_map[i],i)
            
        
get_emotion_for_each_song(data_map=data_map, st_year=start_year+1, ed_year=end_year)


c:\Users\16462\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.us-south.natural-language-understanding.watson.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Poker Face  No data of lyrics


c:\Users\16462\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.us-south.natural-language-understanding.watson.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\16462\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.us-south.natural-language-understanding.watson.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\16462\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.us-south.natural-language-underst

Live Your Life  No data of lyrics


c:\Users\16462\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.us-south.natural-language-understanding.watson.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\16462\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.us-south.natural-language-understanding.watson.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\16462\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.us-south.natural-language-underst

...

c:\Users\16462\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.us-south.natural-language-understanding.watson.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\16462\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.us-south.natural-language-understanding.watson.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\16462\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.us-south.natural-language-underst

Bang Bang  No data of lyrics


c:\Users\16462\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.us-south.natural-language-understanding.watson.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\16462\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.us-south.natural-language-understanding.watson.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\16462\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.us-south.natural-language-underst

Best Day Of My Life  No data of lyrics


c:\Users\16462\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.us-south.natural-language-understanding.watson.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\16462\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.us-south.natural-language-understanding.watson.cloud.ibm.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\16462\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.us-south.natural-language-underst